<a href="https://colab.research.google.com/github/robbyturnip/machine_learning/blob/master/sentiment_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import re
import csv
import json
import pprint
import pickle
import pandas as pd
import nltk.classify
from nltk.tokenize import word_tokenize 

In [19]:
def replaceTwoOrMore(s):
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL) 
    return pattern.sub(r"\1\1", s)

def processTweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    tweet = re.sub('@[^\s]+','AT_USER',tweet)    
    tweet = re.sub('[\s]+', ' ', tweet)
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    tweet = tweet.strip('\'"')
    return tweet

def getStopWordList(stopWordListFileName):
    stopWords = []
    stopWords.append('AT_USER')
    stopWords.append('URL')

    fp = open(stopWordListFileName, 'r')
    line = fp.readline()
    while line:
        word = line.strip()
        stopWords.append(word)
        line = fp.readline()
    fp.close()
    
    return stopWords

def getFeatureVector(tweet, stopWords):
    featureVector = []  
    words = tweet.split()
    for w in words:
        w = replaceTwoOrMore(w) 
        w = w.strip('\'"?,.')
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*[a-zA-Z]+[a-zA-Z0-9]*$", w)
        if(w in stopWords or val is None):
            continue
        else:
            featureVector.append(w.lower())
    return featureVector    

def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in featureList:
        features['contains(%s)' % word] = (word in tweet_words)
    return features

def getSVMFeatureVectorAndLabels(tweets, featureList):
    feature_vector = []
    sentiment = []
    for t in tweets:
        print(t)
        sentiment = 0
        tweet_words = t[0]
        tweet_opinion = t[1]
        for word in tweet_words:
            word = replaceTwoOrMore(word) 
            word = word.strip('\'"?,.')
        values = map.values()
        feature_vector.append(values)
        if(tweet_opinion == 'positive'):
            sentiment = 0
        elif(tweet_opinion == 'negative'):
            sentiment = 1
        elif(tweet_opinion == 'neutral'):
            sentiment = 2
        sentiment.append(sentiment)

    return {'feature_vector' : feature_vector, 'sentiment': sentiment}

inpTweets = csv.reader(open('datatrain.csv', 'r'), delimiter=',')
stopWords = getStopWordList('stopwordsID.txt')
count = 0;
featureList = []
tweets = []
list_data = []

for index,row in enumerate(inpTweets):
    
    if index == 0:
        pass
    
    elif index <= 1001 :
        sentiment = row[1]
        tweet = row[2]
        tweet = processTweet(tweet)
        mydata = (tweet,sentiment)
        tweet = getFeatureVector(tweet,stopWords)
        tweet = ' '.join(tweet)
        mydata = (tweet,sentiment)
        list_data.append(mydata)
        
    else :
        pass

json_dict = {}
all_words = set(word.lower() for passage in list_data for word in word_tokenize(passage[0]))
json_dict['corpus'] = [word for word in all_words]

with open('corpus.json', 'w') as outfile:
    json.dump(json_dict, outfile)
    
t = [({word.lower(): (word.lower() in word_tokenize(x[0].lower())) for word in all_words}, x[1].lower()) for x in list_data]
classifier = nltk.NaiveBayesClassifier.train(t)
f = open('sentiment.pickle', 'wb')
pickle.dump(classifier, f)
f.close()

In [26]:
f = open('sentiment.pickle', 'rb')
classifier = pickle.load(f)
f.close()


d = open('corpus.json')
all_words = json.load(d)
d.close

list_tweet =  []
list_kelas = []
list_data = []

data_test = csv.reader(open('datatest.csv', 'r'), delimiter=',')
for data in data_test:
    list_data.append(data)

for data in list_data[:100]:
    tweet = data[0]
    tweet = processTweet(tweet)
    tweet = getFeatureVector(tweet,stopWords)
    tweet = ' '.join(tweet)
    test_sent_features = {word.lower(): (word in word_tokenize(tweet.lower())) for word in all_words}
    sentiment = classifier.classify(test_sent_features)
    list_tweet.append(data[0])
    list_kelas.append(sentiment)

df = pd.DataFrame({'tweet': list_tweet,'kelas': list_kelas})
df.to_csv('hasiltest.csv', sep=';', encoding='utf-8', index=False)
test_set = csv.reader(open('hasiltest.csv', 'r'), delimiter=';')
data_list = []

for index,data in enumerate(test_set):
    if index == 0:
        pass
    elif index < 101 :
        sentiment = data[1]
        tweet = data[0]
        tweet = processTweet(tweet)
        tweet = getFeatureVector(tweet,stopWords)
        tweet = ' '.join(tweet)
        mydata = (tweet,sentiment)
        data_list.append(mydata)
    else:
        pass

all_words = all_words['corpus']

test_set = [({word.lower() : (word.lower() in word_tokenize(x[0])) for word in all_words}, x[1]) for x in data_list]
accuracy = (nltk.classify.util.accuracy(classifier, test_set))*100
print(accuracy)


12.0


In [27]:
feature = classifier.show_most_informative_features(10)

Most Informative Features
                   ritel = True           negati : positi =     11.3 : 1.0
                   tutup = True           negati : positi =     11.0 : 1.0
                    lesu = True           negati : positi =      8.2 : 1.0
                   orang = True           positi : negati =      8.1 : 1.0
                   utang = True           negati : positi =      7.3 : 1.0
                    toko = True           negati : positi =      6.8 : 1.0
                  kredit = True           negati : positi =      6.8 : 1.0
                   turun = True           negati : positi =      6.7 : 1.0
                   mahal = True           negati : positi =      6.5 : 1.0
              penjelasan = True           negati : positi =      6.5 : 1.0
